In [28]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(style="ticks")
import matplotlib.pyplot as plt
import multiprocessing
from joblib import Parallel, delayed
%matplotlib inline
from scipy.optimize import curve_fit 
from scipy import stats
import datetime as dt
import statsmodels.api as sm
from pandas.plotting import scatter_matrix
from sklearn.linear_model import LinearRegression
from scipy.stats import skew
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
import pickle

In [2]:
#df_main = pd.read_pickle('/home/d.dasarathan/project_sml/df_cleaned_data_nonulls.pkl')
#df_main = pd.read_pickle('/home/d.dasarathan/project_sml/100Kdataset/df_cleaned_data_nonulls.pkl')
df_main = pd.read_pickle('/home/d.dasarathan/project_sml/100Kdataset/df_cleaned_data_nonulls_500k.pkl')


#### Calculate job wait time (Target variable) based on when the job was ready to be put in queue

In [3]:
df_main['waitTime'] = df_main.Start - df_main.Submit
df_main['waitTime']

0        0 days 00:19:56
1        0 days 00:05:39
2        0 days 00:00:14
3        0 days 00:12:55
4        1 days 07:09:28
               ...      
499993   0 days 00:04:45
499994   0 days 00:01:29
499995   0 days 00:01:06
499996   0 days 00:00:14
499997   0 days 00:23:31
Name: waitTime, Length: 499998, dtype: timedelta64[ns]

In [4]:
df_main['waitTimeHr'] = df_main.waitTime / np.timedelta64(1, 'h')
df_main['waitTimeHr']

0          0.332222
1          0.094167
2          0.003889
3          0.215278
4         31.157778
            ...    
499993     0.079167
499994     0.024722
499995     0.018333
499996     0.003889
499997     0.391944
Name: waitTimeHr, Length: 499998, dtype: float64

In [5]:
# Extract int value and char unit from ReqMem column
df_main[['ReqMem_INT', 'ReqMem_unit']] = df_main['ReqMem'].str.extract('(\d+\.?\d*)([A-Za-z]*)', expand = True)

In [6]:
df_main.ReqMem_INT = df_main.ReqMem_INT.astype(float)
#df_main.ReqNodes = df_main.ReqNodes.astype(int)
df_main.ReqNodes = df_main.ReqNodes.astype(str)
df_main.ReqNodes = df_main.ReqNodes.str.replace(r'\D', '').astype(int)
df_main.ReqCPUS = df_main.ReqCPUS.astype(int)
df_main.NCPUS = df_main.NCPUS.astype(int)

<ipython-input-6-c3482ff0ab63>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_main.ReqNodes = df_main.ReqNodes.str.replace(r'\D', '').astype(int)


In [7]:
# Create a new column that has total mem requested in a job
df_main['Req_totalMem'] = pd.Series()

<ipython-input-7-5f9576da7a38>:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_main['Req_totalMem'] = pd.Series()


In [8]:
df_main['Req_totalMem'] = np.where (df_main.ReqMem_unit == 'Gn', (df_main.ReqMem_INT * df_main.ReqNodes),df_main['Req_totalMem'])
df_main['Req_totalMem'] = np.where (df_main.ReqMem_unit == 'Gc' , (df_main.ReqMem_INT * df_main.ReqCPUS), df_main['Req_totalMem'])
df_main['Req_totalMem'] = np.where (df_main.ReqMem_unit == 'Mc' , ((df_main.ReqMem_INT * df_main.ReqCPUS)/1024) , df_main['Req_totalMem'])
df_main['Req_totalMem'] = np.where (df_main.ReqMem_unit == 'Mn', ((df_main.ReqMem_INT * df_main.ReqNodes)/1024) ,df_main['Req_totalMem'])
df_main['Req_totalMem'] = np.where (df_main.ReqMem_unit == 'n' , 0 , df_main['Req_totalMem'])
df_main['Req_totalMem'] = np.where (df_main.ReqMem_unit == 'Tn', (df_main.ReqMem_INT * df_main.ReqNodes * 1024) ,df_main['Req_totalMem'])
df_main['Req_totalMem'] = np.where (df_main.ReqMem_unit == 'Tc' , (df_main.ReqMem_INT * df_main.ReqCPUS * 1024), df_main['Req_totalMem'])

In [9]:
# Split days and duration into two columns and then create a new measure that has hours (sum of both columns)
df_main['TLdays']=np.where(df_main.Timelimit.str.contains("-"),df_main['Timelimit'].str.split('-').str[0],'0')

# Change the data type from str to int
df_main['TLdays']=pd.to_numeric(df_main.TLdays)

# Replace "Partition Limit" & "UNLIMITED" values in the time limit column with "23:59:59"
df_main['TLduration']=np.where(df_main.Timelimit.str.contains("-"),df_main['Timelimit'].str.split('-').str[1],df_main['Timelimit'])
df_main.TLduration=np.where(df_main.TLduration.str.contains('Partition_Limit'), "23:59:59", df_main.TLduration)
df_main.TLduration=np.where(df_main.TLduration.str.contains('UNLIMITED'), "23:59:59", df_main.TLduration)

# Change the data type from string to time
df_main.TLduration=pd.to_timedelta(df_main.TLduration)

In [10]:
# Calculate one field from TLdays & TLduration and drop the rest (not required) 
df_main['Timelimit_hr'] = (df_main.TLdays * 24) + (df_main.TLduration / np.timedelta64(1, 'h'))

In [11]:
# Change the value of partition where it contains a "," Example: 'short,large,express,west', 'short,reservation', 'short,express'
df_main.Partition=np.where(df_main.Partition.str.contains(','), "partcombo", df_main.Partition)

In [12]:
# Drop unnecessary columns (derived new features using these)
df_main=df_main.drop(['ReqMem','ReqMem_INT','ReqMem_unit','TLdays','TLduration','Timelimit'], axis=1)

In [13]:
# Fill the nan values in priority column with mean priority so that the input does not introduce bias.
df_main['Priority']=df_main['Priority'].fillna((df_main['Priority'].mean()))

In [14]:
# Creating a new column using submit time of the job. The new column tells us in which quarter of the day the job was submitted
def getquartofday(dtcol):
    QODcol = dtcol.dt.strftime("%H")
    QODcol = np.where((QODcol == '00') | (QODcol == '01') | (QODcol == '02') | (QODcol == '03') | (QODcol == '04') | (QODcol == '05'), 'q1', QODcol)
    QODcol = np.where((QODcol == '06') | (QODcol == '07') | (QODcol == '08') | (QODcol == '09') | (QODcol == '10') | (QODcol == '11'), 'q2', QODcol)
    QODcol = np.where((QODcol == '12') | (QODcol == '13') | (QODcol == '14') | (QODcol == '15') | (QODcol == '16') | (QODcol == '17'), 'q3', QODcol)
    QODcol = np.where((QODcol == '18') | (QODcol == '19') | (QODcol == '20') | (QODcol == '21') | (QODcol == '22') | (QODcol == '23'), 'q4', QODcol)
    return QODcol

df_main['QOD'] = getquartofday(df_main.Submit)

In [15]:
# Creating a new column using submit time of the job. The new column tells us in which quarter of the year the job was submitted
def getquartofyear(dtcol):
    QOYcol = dtcol.dt.strftime("%b")
    QOYcol = np.where((QOYcol == 'Jan') | (QOYcol == 'Feb') | (QOYcol == 'Mar'), 'q1', QOYcol)
    QOYcol = np.where((QOYcol == 'Apr') | (QOYcol == 'May') | (QOYcol == 'Jun'), 'q2', QOYcol)
    QOYcol = np.where((QOYcol == 'Jul') | (QOYcol == 'Aug') | (QOYcol == 'Sep'), 'q3', QOYcol)
    QOYcol = np.where((QOYcol == 'Oct') | (QOYcol == 'Nov') | (QOYcol == 'Dec'), 'q4', QOYcol)
    return QOYcol

df_main['QOY'] = getquartofyear(df_main.Submit)

In [16]:
# Create exlusive flag just like the slurm parameter
df_main['exclusive']=np.where(df_main.NCPUS != df_main.ReqCPUS, 1, 0)

In [17]:
# Calculate total core hours
df_main['corehrs']= df_main.Timelimit_hr * df_main.ReqCPUS

In [18]:
#df_main_req = df_main[['ReqNodes', 'Start', 'End', 'Submit', 'Priority', 'Partition', 'waitTimeHr', 'Req_totalMem', 'corehrs', 'QOD', 'QOY', 'exclusive']]
#df_main_req.head()

In [19]:
def calc_rnjobct(rowsubmit, start, end):
    return len(np.where((rowsubmit > start) & (rowsubmit < end))[0])


In [20]:
import swifter
import time
start_time = time.time()
df_main['activejobcount'] = df_main[['Submit','Start','End']].swifter.apply(lambda row: calc_rnjobct(row['Submit'], df_main.Start, df_main.End), axis=1)

print("--- %s seconds ---" % (time.time() - start_time))

Dask Apply:   0%|          | 0/256 [00:00<?, ?it/s]

--- 2467.7010951042175 seconds ---


In [21]:
#df_main_req.to_pickle('/scratch/d.dasarathan/df_main1_first4M.pkl')
#df_main_req.head()
df_main.head()

,Account,AdminComment,AllocCPUS,AllocGRES,AllocNodes,AllocTRES,AssocID,AveCPU,AveCPUFreq,AveDiskRead,AveDiskWrite,AvePages,AveRSS,AveVMSize,BlockID,Cluster,Comment,ConsumedEnergy,ConsumedEnergyRaw,CPUTime,CPUTimeRAW,DerivedExitCode,Elapsed,ElapsedRaw,Eligible,End,ExitCode,GID,Group,JobID,JobIDRaw,JobName,Layout,MaxDiskRead,MaxDiskReadNode,MaxDiskReadTask,MaxDiskWrite,MaxDiskWriteNode,MaxDiskWriteTask,MaxPages,MaxPagesNode,MaxPagesTask,MaxRSS,MaxRSSNode,MaxRSSTask,MaxVMSize,MaxVMSizeNode,MaxVMSizeTask,McsLabel,MinCPU,MinCPUNode,MinCPUTask,NCPUS,NNodes,NodeList,NTasks,Priority,Partition,QOS,QOSRAW,ReqCPUFreq,ReqCPUFreqMin,ReqCPUFreqMax,ReqCPUFreqGov,ReqCPUS,ReqGRES,ReqNodes,ReqTRES,Reservation,ReservationId,Reserved,ResvCPU,ResvCPURAW,Start,State,Submit,Suspended,SystemCPU,TotalCPU,UID,User,UserCPU,WCKey,WCKeyID,WorkDir,Unnamed: 87,waitTime,waitTimeHr,Req_totalMem,Timelimit_hr,QOD,QOY,exclusive,corehrs,rnjbct
0,steven.lopez,NaN,7.0,NaN,1.0,"cpu=7,mem=55G,node=1,billing=7",5187,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discovery,NaN,0.0,0.0,00:40:36,2436.0,0:0,00:05:48,348,2020-05-17 15:35:54,2020-05-17 16:01:36,0:0,100.0,users,11602102_128,11602922,cp-4e4r-ORCA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,1.0,d3004,NaN,16007.0,lopez,normal,1.0,Unknown,Unknown,Unknown,Unknown,7,NaN,1,"cpu=7,mem=55G,node=1",NaN,NaN,00:19:54,02:19:18,8358.0,2020-05-17 15:55:48,COMPLETED,2020-05-17 15:35:52,00:00:00,00:09.466,29:29.533,1.825619e+09,steinr,29:20.066,NaN,0.0,/scratch/steinr/ORCA/tod/opt/cp-4e4r/CREST,NaN,0 days 00:19:56,0.332222,55.0,24.0,q3,q2,0,168.0,77
1,lotterhos,NaN,1.0,NaN,1.0,"cpu=1,mem=50G,node=1,billing=1",4488.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discovery,NaN,0.0,0.0,00:01:35,95.0,0:0,00:01:35,95,2020-08-13 17:14:24,2020-08-13 17:21:38,0:0,100.0,users,13253497,13253497,3383.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,c0198,NaN,17612.0,express,normal,1.0,Unknown,Unknown,Unknown,Unknown,1,NaN,10,"cpu=1,mem=50G,node=1",NaN,NaN,00:05:39,00:05:39,339.0,2020-08-13 17:20:03,COMPLETED,2020-08-13 17:14:24,00:00:00,00:00.540,01:34.502,1.825568e+09,albecker,01:33.961,NaN,0.0,/scratch/albecker/Power_analysis,NaN,0 days 00:05:39,0.094167,500.0,1.0,q3,q3,0,1.0,151
2,dapeng,NaN,28.0,NaN,1.0,"cpu=28,mem=200G,node=1,billing=28",5831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discovery,NaN,0.0,0.0,00:22:52,1372.0,0:0,00:00:49,49,2020-09-08 00:40:55,2020-09-08 00:41:58,0:0,100.0,users,13946485,13946485,short_22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28,1.0,c0476,NaN,15077.0,short,normal,1.0,Unknown,Unknown,Unknown,Unknown,28,NaN,1,"cpu=28,mem=200G,node=1",NaN,NaN,00:00:14,00:06:32,392.0,2020-09-08 00:41:09,COMPLETED,2020-09-08 00:40:55,00:00:00,00:02.827,00:18.500,1.825455e+09,huang.junx,00:15.673,NaN,0.0,/scratch/jxh,NaN,0 days 00:00:14,0.003889,200.0,24.0,q1,q3,0,672.0,82
3,steven.lopez,NaN,1.0,NaN,1.0,"cpu=1,mem=3G,node=1,billing=1",4525.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discovery,NaN,0.0,0.0,00:07:29,449.0,0:0,00:07:29,449,2020-08-06 17:10:51,2020-08-06 17:31:15,0:0,100.0,users,13030602,13030602,2bt-83-81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,c0308,NaN,15738.0,short,normal,1.0,Unknown,Unknown,Unknown,Unknown,1,NaN,10,"cpu=1,mem=3G,node=1",NaN,NaN,00:12:55,00:12:55,775.0,2020-08-06 17:23:46,COMPLETED,2020-08-06 17:10:51,00:00:00,00:59.926,07:19.510,1.825607e+09,lijingbai2009,06:19.584,NaN,0.0,/scratch/lijingbai2009/NN-PES-QD/training-data...,NaN,0 days 00:12:55,0.215278,30.0,10.0,q3,q3,0,10.0,64
4,netsi,NaN,1.0,NaN,1.0,"cpu=1,mem=200G,node=1,billing=1",5217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discovery,NaN,0.0,0.0,00:00:03,3.0,0:0,00:00:03,3,2020-09-11 19:11:53,2020-09-13 02:21:24,1:0,100.0,users,14255256,14255256,un_eval,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,c3201,NaN,16757.0,netsi_standard,normal,1.0,Unknown,Unknown,Unknown,Unknown,1,NaN,1,"cpu=1,me

In [22]:
# Reset index
#df_main_req=df_main_req.reset_index(drop=True)
#df_main_req.head()
df_main=df_main.reset_index(drop=True)
df_main.head()

,Account,AdminComment,AllocCPUS,AllocGRES,AllocNodes,AllocTRES,AssocID,AveCPU,AveCPUFreq,AveDiskRead,AveDiskWrite,AvePages,AveRSS,AveVMSize,BlockID,Cluster,Comment,ConsumedEnergy,ConsumedEnergyRaw,CPUTime,CPUTimeRAW,DerivedExitCode,Elapsed,ElapsedRaw,Eligible,End,ExitCode,GID,Group,JobID,JobIDRaw,JobName,Layout,MaxDiskRead,MaxDiskReadNode,MaxDiskReadTask,MaxDiskWrite,MaxDiskWriteNode,MaxDiskWriteTask,MaxPages,MaxPagesNode,MaxPagesTask,MaxRSS,MaxRSSNode,MaxRSSTask,MaxVMSize,MaxVMSizeNode,MaxVMSizeTask,McsLabel,MinCPU,MinCPUNode,MinCPUTask,NCPUS,NNodes,NodeList,NTasks,Priority,Partition,QOS,QOSRAW,ReqCPUFreq,ReqCPUFreqMin,ReqCPUFreqMax,ReqCPUFreqGov,ReqCPUS,ReqGRES,ReqNodes,ReqTRES,Reservation,ReservationId,Reserved,ResvCPU,ResvCPURAW,Start,State,Submit,Suspended,SystemCPU,TotalCPU,UID,User,UserCPU,WCKey,WCKeyID,WorkDir,Unnamed: 87,waitTime,waitTimeHr,Req_totalMem,Timelimit_hr,QOD,QOY,exclusive,corehrs,rnjbct
0,steven.lopez,NaN,7.0,NaN,1.0,"cpu=7,mem=55G,node=1,billing=7",5187,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discovery,NaN,0.0,0.0,00:40:36,2436.0,0:0,00:05:48,348,2020-05-17 15:35:54,2020-05-17 16:01:36,0:0,100.0,users,11602102_128,11602922,cp-4e4r-ORCA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,1.0,d3004,NaN,16007.0,lopez,normal,1.0,Unknown,Unknown,Unknown,Unknown,7,NaN,1,"cpu=7,mem=55G,node=1",NaN,NaN,00:19:54,02:19:18,8358.0,2020-05-17 15:55:48,COMPLETED,2020-05-17 15:35:52,00:00:00,00:09.466,29:29.533,1.825619e+09,steinr,29:20.066,NaN,0.0,/scratch/steinr/ORCA/tod/opt/cp-4e4r/CREST,NaN,0 days 00:19:56,0.332222,55.0,24.0,q3,q2,0,168.0,77
1,lotterhos,NaN,1.0,NaN,1.0,"cpu=1,mem=50G,node=1,billing=1",4488.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discovery,NaN,0.0,0.0,00:01:35,95.0,0:0,00:01:35,95,2020-08-13 17:14:24,2020-08-13 17:21:38,0:0,100.0,users,13253497,13253497,3383.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,c0198,NaN,17612.0,express,normal,1.0,Unknown,Unknown,Unknown,Unknown,1,NaN,10,"cpu=1,mem=50G,node=1",NaN,NaN,00:05:39,00:05:39,339.0,2020-08-13 17:20:03,COMPLETED,2020-08-13 17:14:24,00:00:00,00:00.540,01:34.502,1.825568e+09,albecker,01:33.961,NaN,0.0,/scratch/albecker/Power_analysis,NaN,0 days 00:05:39,0.094167,500.0,1.0,q3,q3,0,1.0,151
2,dapeng,NaN,28.0,NaN,1.0,"cpu=28,mem=200G,node=1,billing=28",5831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discovery,NaN,0.0,0.0,00:22:52,1372.0,0:0,00:00:49,49,2020-09-08 00:40:55,2020-09-08 00:41:58,0:0,100.0,users,13946485,13946485,short_22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28,1.0,c0476,NaN,15077.0,short,normal,1.0,Unknown,Unknown,Unknown,Unknown,28,NaN,1,"cpu=28,mem=200G,node=1",NaN,NaN,00:00:14,00:06:32,392.0,2020-09-08 00:41:09,COMPLETED,2020-09-08 00:40:55,00:00:00,00:02.827,00:18.500,1.825455e+09,huang.junx,00:15.673,NaN,0.0,/scratch/jxh,NaN,0 days 00:00:14,0.003889,200.0,24.0,q1,q3,0,672.0,82
3,steven.lopez,NaN,1.0,NaN,1.0,"cpu=1,mem=3G,node=1,billing=1",4525.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discovery,NaN,0.0,0.0,00:07:29,449.0,0:0,00:07:29,449,2020-08-06 17:10:51,2020-08-06 17:31:15,0:0,100.0,users,13030602,13030602,2bt-83-81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,c0308,NaN,15738.0,short,normal,1.0,Unknown,Unknown,Unknown,Unknown,1,NaN,10,"cpu=1,mem=3G,node=1",NaN,NaN,00:12:55,00:12:55,775.0,2020-08-06 17:23:46,COMPLETED,2020-08-06 17:10:51,00:00:00,00:59.926,07:19.510,1.825607e+09,lijingbai2009,06:19.584,NaN,0.0,/scratch/lijingbai2009/NN-PES-QD/training-data...,NaN,0 days 00:12:55,0.215278,30.0,10.0,q3,q3,0,10.0,64
4,netsi,NaN,1.0,NaN,1.0,"cpu=1,mem=200G,node=1,billing=1",5217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discovery,NaN,0.0,0.0,00:00:03,3.0,0:0,00:00:03,3,2020-09-11 19:11:53,2020-09-13 02:21:24,1:0,100.0,users,14255256,14255256,un_eval,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,c3201,NaN,16757.0,netsi_standard,normal,1.0,Unknown,Unknown,Unknown,Unknown,1,NaN,1,"cpu=1,me

In [23]:
# Write the cleaned dataset to a pickle file
#df_main_req.to_pickle('/scratch/d.dasarathan/df_newdata_cleaned_added_features.pkl')
#df_main.to_pickle('/scratch/d.dasarathan/df_newdata_cleaned_100k_added_features.pkl')
df_main.to_pickle('/scratch/d.dasarathan/df_newdata_cleaned_500k_added_features.pkl')

In [34]:
#df_sbatch = pd.read_pickle('/scratch/d.dasarathan/df_newdata_cleaned_added_features_sbatch.pkl')
#df_test = pd.read_pickle('/scratch/d.dasarathan/df_newdata_cleaned_100k_added_features.pkl')
df_test = pd.read_pickle('/scratch/d.dasarathan/df_newdata_cleaned_500k_added_features.pkl')

In [35]:
df_test.head()

,Account,AdminComment,AllocCPUS,AllocGRES,AllocNodes,AllocTRES,AssocID,AveCPU,AveCPUFreq,AveDiskRead,AveDiskWrite,AvePages,AveRSS,AveVMSize,BlockID,Cluster,Comment,ConsumedEnergy,ConsumedEnergyRaw,CPUTime,CPUTimeRAW,DerivedExitCode,Elapsed,ElapsedRaw,Eligible,End,ExitCode,GID,Group,JobID,JobIDRaw,JobName,Layout,MaxDiskRead,MaxDiskReadNode,MaxDiskReadTask,MaxDiskWrite,MaxDiskWriteNode,MaxDiskWriteTask,MaxPages,MaxPagesNode,MaxPagesTask,MaxRSS,MaxRSSNode,MaxRSSTask,MaxVMSize,MaxVMSizeNode,MaxVMSizeTask,McsLabel,MinCPU,MinCPUNode,MinCPUTask,NCPUS,NNodes,NodeList,NTasks,Priority,Partition,QOS,QOSRAW,ReqCPUFreq,ReqCPUFreqMin,ReqCPUFreqMax,ReqCPUFreqGov,ReqCPUS,ReqGRES,ReqNodes,ReqTRES,Reservation,ReservationId,Reserved,ResvCPU,ResvCPURAW,Start,State,Submit,Suspended,SystemCPU,TotalCPU,UID,User,UserCPU,WCKey,WCKeyID,WorkDir,Unnamed: 87,waitTime,waitTimeHr,Req_totalMem,Timelimit_hr,QOD,QOY,exclusive,corehrs,rnjbct
0,steven.lopez,NaN,7.0,NaN,1.0,"cpu=7,mem=55G,node=1,billing=7",5187,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discovery,NaN,0.0,0.0,00:40:36,2436.0,0:0,00:05:48,348,2020-05-17 15:35:54,2020-05-17 16:01:36,0:0,100.0,users,11602102_128,11602922,cp-4e4r-ORCA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,1.0,d3004,NaN,16007.0,lopez,normal,1.0,Unknown,Unknown,Unknown,Unknown,7,NaN,1,"cpu=7,mem=55G,node=1",NaN,NaN,00:19:54,02:19:18,8358.0,2020-05-17 15:55:48,COMPLETED,2020-05-17 15:35:52,00:00:00,00:09.466,29:29.533,1.825619e+09,steinr,29:20.066,NaN,0.0,/scratch/steinr/ORCA/tod/opt/cp-4e4r/CREST,NaN,0 days 00:19:56,0.332222,55.0,24.0,q3,q2,0,168.0,77
1,lotterhos,NaN,1.0,NaN,1.0,"cpu=1,mem=50G,node=1,billing=1",4488.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discovery,NaN,0.0,0.0,00:01:35,95.0,0:0,00:01:35,95,2020-08-13 17:14:24,2020-08-13 17:21:38,0:0,100.0,users,13253497,13253497,3383.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,c0198,NaN,17612.0,express,normal,1.0,Unknown,Unknown,Unknown,Unknown,1,NaN,10,"cpu=1,mem=50G,node=1",NaN,NaN,00:05:39,00:05:39,339.0,2020-08-13 17:20:03,COMPLETED,2020-08-13 17:14:24,00:00:00,00:00.540,01:34.502,1.825568e+09,albecker,01:33.961,NaN,0.0,/scratch/albecker/Power_analysis,NaN,0 days 00:05:39,0.094167,500.0,1.0,q3,q3,0,1.0,151
2,dapeng,NaN,28.0,NaN,1.0,"cpu=28,mem=200G,node=1,billing=28",5831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discovery,NaN,0.0,0.0,00:22:52,1372.0,0:0,00:00:49,49,2020-09-08 00:40:55,2020-09-08 00:41:58,0:0,100.0,users,13946485,13946485,short_22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28,1.0,c0476,NaN,15077.0,short,normal,1.0,Unknown,Unknown,Unknown,Unknown,28,NaN,1,"cpu=28,mem=200G,node=1",NaN,NaN,00:00:14,00:06:32,392.0,2020-09-08 00:41:09,COMPLETED,2020-09-08 00:40:55,00:00:00,00:02.827,00:18.500,1.825455e+09,huang.junx,00:15.673,NaN,0.0,/scratch/jxh,NaN,0 days 00:00:14,0.003889,200.0,24.0,q1,q3,0,672.0,82
3,steven.lopez,NaN,1.0,NaN,1.0,"cpu=1,mem=3G,node=1,billing=1",4525.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discovery,NaN,0.0,0.0,00:07:29,449.0,0:0,00:07:29,449,2020-08-06 17:10:51,2020-08-06 17:31:15,0:0,100.0,users,13030602,13030602,2bt-83-81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,c0308,NaN,15738.0,short,normal,1.0,Unknown,Unknown,Unknown,Unknown,1,NaN,10,"cpu=1,mem=3G,node=1",NaN,NaN,00:12:55,00:12:55,775.0,2020-08-06 17:23:46,COMPLETED,2020-08-06 17:10:51,00:00:00,00:59.926,07:19.510,1.825607e+09,lijingbai2009,06:19.584,NaN,0.0,/scratch/lijingbai2009/NN-PES-QD/training-data...,NaN,0 days 00:12:55,0.215278,30.0,10.0,q3,q3,0,10.0,64
4,netsi,NaN,1.0,NaN,1.0,"cpu=1,mem=200G,node=1,billing=1",5217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,discovery,NaN,0.0,0.0,00:00:03,3.0,0:0,00:00:03,3,2020-09-11 19:11:53,2020-09-13 02:21:24,1:0,100.0,users,14255256,14255256,un_eval,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,c3201,NaN,16757.0,netsi_standard,normal,1.0,Unknown,Unknown,Unknown,Unknown,1,NaN,1,"cpu=1,me

In [36]:
df_test.shape

(499998, 95)

In [37]:
df_test.describe()

,AdminComment,AllocCPUS,AllocNodes,AveCPU,AveCPUFreq,AveDiskRead,AveDiskWrite,AvePages,AveRSS,AveVMSize,BlockID,ConsumedEnergy,CPUTimeRAW,Layout,MaxDiskRead,MaxDiskReadNode,MaxDiskReadTask,MaxDiskWrite,MaxDiskWriteNode,MaxDiskWriteTask,MaxPages,MaxPagesNode,MaxPagesTask,MaxRSS,MaxRSSNode,MaxRSSTask,MaxVMSize,MaxVMSizeNode,MaxVMSizeTask,McsLabel,MinCPU,MinCPUNode,MinCPUTask,NCPUS,NNodes,NTasks,Priority,QOSRAW,ReqCPUS,ReqNodes,ReservationId,ResvCPURAW,UID,WCKeyID,Unnamed: 87,waitTime,waitTimeHr,Req_totalMem,Timelimit_hr,exclusive,corehrs,rnjbct
count,0.0,499998.000000,499998.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,478312.0,4.999980e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,499998.000000,499998.000000,0.0,499998.000000,499998.000000,499998.000000,499998.000000,4596.000000,4.999980e+05,4.972990e+05,499998.0,0.0,499998,499998.000000,499998.000000,499998.000000,499998.000000,499998.000000,499998.000000
mean,NaN,17.792475,1.160699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,7.113651e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.792475,1.207487,NaN,13526.344791,1.000038,16.408146,6.041722,190.710183,1.920688e+05,8.325752e+08,0.0,NaN,0 days 03:41:15.941617766,3.687762,198.528650,34.381816,0.046268,272.548070,99.497302
std,NaN,34.864496,1.242723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,8.759053e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.864496,1.249304,NaN,5822.058566,0.026870,32.585952,8.371839,29.886160,2.018240e+07,9.092096e+08,0.0,NaN,0 days 22:35:52.516031082,22.597921,584.230874,305.942027,0.210066,1331.492807,50.271941
min,NaN,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,127.000000,0.000000e+00,0.000000e+00,0.0,NaN,0 days 00:00:00,0.000000,0.000000,0.016667,0.000000,0.016667,0.000000
25%,NaN,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,8.500000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,NaN,13871.000000,1.000000,1.000000,1.000000,183.000000,5.300000e+01,5.092800e+04,0.0,NaN,0 days 00:00:24,0.006667,10.000000,4.000000,0.000000,6.000000,64.000000
50%,NaN,7.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,5.720000e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.000000,1.000000,NaN,15724.000000,1.000000,4.000000,2.000000,183.000000,5.880000e+02,5.128500e+04,0.0,NaN,0 days 00:01:56,0.032222,60.000000,12.000000,0.000000,48.000000,84.000000
75%,NaN,28.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,6.197000e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.000000,1.000000,NaN,16498.000000,1.000000,28.000000,10.000000,216.000000,5.214000e+03,1.825548e+09,0.0,NaN,0 days 00:35:40,0.594444,200.000000,24.000000,0.000000,336.000000,118.000000
max,NaN,3136.000000,256.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.832782e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3136.000000,256.000000,NaN,23732.000000,20.000000,3136.000000,900.000000,252.000000,4.294967e+09,1.825729e+09,0.0,NaN,174 days 19:13:11,4195.219722,57600.000000,8760.000000,1.000000,268800.000000,390.000000


In [38]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499998 entries, 0 to 499997
Data columns (total 95 columns):
 #   Column             Non-Null Count   Dtype          
---  ------             --------------   -----          
 0   Account            499998 non-null  object         
 1   AdminComment       0 non-null       float64        
 2   AllocCPUS          499998 non-null  float64        
 3   AllocGRES          16962 non-null   object         
 4   AllocNodes         499998 non-null  float64        
 5   AllocTRES          479025 non-null  object         
 6   AssocID            499998 non-null  object         
 7   AveCPU             0 non-null       float64        
 8   AveCPUFreq         0 non-null       float64        
 9   AveDiskRead        0 non-null       float64        
 10  AveDiskWrite       0 non-null       float64        
 11  AvePages           0 non-null       float64        
 12  AveRSS             0 non-null       float64        
 13  AveVMSize          0 non-null